You are to build upon the predictive analysis (classification) that you already completed in the previous mini-project, adding additional modeling from new classification algorithms as well as more explanations that are inline with the CRISP-DM framework. You should use appropriate cross validation for all of your analysis (explain your chosen method of performance validation in detail). Try to use as much testing data as possible in a realistic manner (you should define what you think is realistic and why).
This report is worth 20% of the final grade. Please upload a report (one per team) with all code used, visualizations, and text in a single document. The format of the document can be PDF, *.ipynb, or HTML. You can write the report in whatever format you like, but it is easiest to turn in the rendered iPython notebook. The results should be reproducible using your report. Please carefully describe every assumption and every step in your report.

### Dataset Selection
Select a dataset identically to the way you selected for the first project work week and mini-project. You are not required to use the same dataset that you used in the past, but you are encouraged. You must identify two tasks from the dataset to regress or classify. That is:
- two classification tasks OR
- two regression tasks OR
- one classification task and one regression task

For example, if your dataset was from the diabetes data you might try to predict two tasks: (1) classifying if a patient will be readmitted within a 30 day period or not, and (2) regressing what the total number of days a patient will spend in the hospital, given their history and specifics of the encounter like tests administered and previous admittance.

### Grading Rubric

#### Data Preparation (15 points total)
- [10 points] Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis.
 
- [5 points] Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).
### Modeling and Evaluation (70 points total)
- [10 points] Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.

- [10 points] Choose the method you will use for dividing your data into training and testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate.
- [20 points] Create three different classification/regression models (e.g., random forest, KNN, and SVM). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric.
- [10 points] Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model.
- [10 points] Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods.
- [10 points] Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task.
### Deployment (5 points total)
- [5 points] How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.?
### Exceptional Work (10 points total)
- You have free reign to provide additional modeling.
- One idea: grid search parameters in a parallelized fashion and visualize the performances across attributes. Which parameters are most significant for making a good model for each classification algorithm?

Two dataframes for each classification task

Data cleanup (Dylan and Satvik)
Broad phase of flight dataframe

Injury (Injury)  for KNN (Nnenna)
- Look into ROC Curves
- Look at Sklearn parameters for KNN


Injury (Injury) for Decision Trees (Jobin)
- Look at Sklearn parameters for decision trees

Injury (Injury) for KNN

Injury (Injury) for Decision Trees



In [297]:
import pandas as pd
import numpy as np

In [298]:
#Read in the Aviation Data
final_data = pd.read_csv("../Data/final_data.csv",low_memory=False,dtype={'damage': str})
#Delete columns that were imported incorrectly
del final_data["Unnamed: 0"]
del final_data["dprt_state.1"]
del final_data["index"]
del final_data["ntsb_no_x"]
del final_data['wind_vel_ind']

final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115706 entries, 0 to 115705
Data columns (total 30 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ev_id              115706 non-null  object 
 1   acft_make          115643 non-null  object 
 2   acft_model         115630 non-null  object 
 3   cert_max_gr_wt     98673 non-null   float64
 4   acft_category      115287 non-null  object 
 5   damage             113877 non-null  object 
 6   far_part           114925 non-null  object 
 7   afm_hrs_last_insp  60298 non-null   float64
 8   type_fly           108599 non-null  object 
 9   dprt_city          111864 non-null  object 
 10  dprt_state         108791 non-null  object 
 11  rwy_len            64222 non-null   float64
 12  rwy_width          63110 non-null   float64
 13  ev_type            115706 non-null  object 
 14  ev_city            115646 non-null  object 
 15  ev_state           109635 non-null  object 
 16  ev

In [299]:
pd.set_option('display.max_columns', 50)

In [300]:
final_data.head(10)

,ev_id,acft_make,acft_model,cert_max_gr_wt,acft_category,damage,far_part,afm_hrs_last_insp,type_fly,dprt_city,dprt_state,rwy_len,rwy_width,ev_type,ev_city,ev_state,ev_country,ev_highest_injury,inj_f_grnd,inj_m_grnd,inj_s_grnd,inj_tot_f,inj_tot_m,inj_tot_n,inj_tot_s,inj_tot_t,sky_cond_ceil,sky_cond_nonceil,wx_int_precip,phase_flt_spec
0,20001204X00000,Cessna,207,3800.0,AIR,SUBS,135,75.0,UNK,BETHEL,AK,NaN,NaN,ACC,QUINHAGAK,AK,USA,MINR,0.0,0.0,0.0,NaN,1.0,NaN,NaN,1.0,BKN,UNK,UNK,Approach
1,20001204X00001,Boeing,747-100,750000.0,AIR,MINR,121,113.0,UNK,CHITOSE,JA,11800.0,150.0,INC,FAIRBANKS,AK,USA,NONE,0.0,0.0,0.0,NaN,NaN,4.0,NaN,NaN,NONE,SCAT,UNK,Landing
2,20001204X00002,Piper,PA-31-350,7369.0,AIR,SUBS,135,32.0,UNK,CHENEGA BAY,AK,NaN,NaN,ACC,ANCHORAGE,AK,USA,NONE,0.0,0.0,0.0,NaN,NaN,6.0,NaN,NaN,OVC,SCAT,UNK,Unknown
3,20001204X00003,Cessna,172,2300.0,AIR,SUBS,091,40.0,PERS,,,6398.0,150.0,ACC,BETHEL,AK,USA,NONE,0.0,0.0,0.0,NaN,NaN,1.0,NaN,NaN,BKN,UNK,LGT,Unknown
4,20001204X00004,Cessna,207,3800.0,AIR,SUBS,135,49.0,UNK,,AK,2610.0,40.0,ACC,CHEVAK,AK,USA,NONE,0.0,0.0,0.0,NaN,NaN,1.0,NaN,NaN,BKN,UNK,UNK,Descent
5,20001204X00005,Piper,PA-22-160,1840.0,AIR,SUBS,091,NaN,PERS,,,2200.0,70.0,ACC,ANCHORAGE,AK,USA,NONE,0.0,0.0,0.0,NaN,NaN,1.0,NaN,NaN,UNK,BKNT,UNK,Takeoff
6,20001204X00006,Beech,300,14100.0,AIR,DEST,091,3.0,EXEC,GREENEVILLE,SC,5500.0,100.0,ACC,CULLMAN,AL,USA,FATL,0.0,0.0,0.0,2.0,NaN,NaN,NaN,2.0,BKN,UNK,MOD,Approach
7,20001204X00007,Piper,PA-28-181,2550.0,AIR,DEST,091,NaN,PERS,ANDREWS,NC,NaN,NaN,ACC,BREVARD,NC,USA,FATL,0.0,0.0,0.0,1.0,NaN,NaN,NaN,1.0,NONE,CLER,UNK,Maneuvering
8,20001204X00008,Aero Commander,560A,6000.0,AIR,DEST,091,13.0,PERS,,,3800.0,36.0,ACC,BELLEVIEW,FL,USA,FATL,0.0,0.0,0.0,2.0,NaN,NaN,2.0,4.0,NONE,CLER,UNK,Approach
9,20001204X00009,Piper,PA-24-250,2900.0,AIR,SUBS,091,40.0,PERS,ALLAIRE,NJ,NaN,NaN,ACC,COLBERT,GA,USA,NONE,0.0,0.0,0.0,NaN,NaN,1.0,NaN,NaN,OVC,UNK,LGT,Unknown


In [301]:
#replace the all empty values to Nan to fix dprt_city column
final_data= final_data.replace(r'^\s+$', np.nan, regex=True)

In [302]:
finaldamagecount = final_data["dprt_city"].value_counts().reset_index()
finaldamagecount

,index,dprt_city
0,Anchorage,393
1,ANCHORAGE,380
2,Houston,226
3,Phoenix,199
4,Fairbanks,196
...,...,...
16399,W. KINGSTON,1
16400,St. Peterburg,1
16401,Sumner,1
16402,WRIGHTSTORM,1


In [303]:
final_data['acft_make'] = final_data['acft_make'].str.upper()
final_data['acft_category'] = final_data['acft_category'].str.upper()
final_data['damage'] = final_data['damage'].str.upper()
final_data['type_fly'] = final_data['type_fly'].str.upper()
final_data['dprt_city'] = final_data['dprt_city'].str.upper()
final_data['dprt_state'] = final_data['dprt_state'].str.upper()
final_data['ev_city'] = final_data['ev_city'].str.upper()
final_data['ev_type'] = final_data['ev_type'].str.upper()
final_data['ev_city'] = final_data['ev_city'].str.upper()
final_data['ev_country'] = final_data['ev_country'].str.upper()
final_data['sky_cond_ceil'] = final_data['sky_cond_ceil'].str.upper()
final_data['sky_cond_nonceil'] = final_data['sky_cond_nonceil'].str.upper()
final_data['wx_int_precip'] = final_data['wx_int_precip'].str.upper()
final_data['phase_flt_spec'] = final_data['phase_flt_spec'].str.upper()
final_data['ev_highest_injury'] = final_data['ev_highest_injury'].str.upper()

In [304]:
final_data.loc[final_data['damage'].str.contains('UNK', na=False), 'damage'] = 'UNK'
final_data.loc[final_data['far_part'].str.contains('UNK', na=False), 'damage'] = 'UNK'
# final_data = final_data.loc[final_data['phase_flt_spec'].str.contains('UNK', na=False), 'damage'] = 'UNK'

In [305]:
percent_missing = final_data.isnull().sum() * 100 / len(final_data)
#Create DF 
missing_value_df = pd.DataFrame({'column_name': final_data.columns,
                                 'percent_missing': percent_missing})
#Then sort by least to most
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
missing_value_df

,column_name,percent_missing
wx_int_precip,wx_int_precip,56.875184
inj_s_grnd,inj_s_grnd,55.473355
inj_m_grnd,inj_m_grnd,55.453477
inj_f_grnd,inj_f_grnd,55.383472
afm_hrs_last_insp,afm_hrs_last_insp,47.886886
inj_tot_s,inj_tot_s,47.528218
rwy_width,rwy_width,45.456588
inj_tot_m,inj_tot_m,44.772095
rwy_len,rwy_len,44.495532
inj_tot_f,inj_tot_f,44.363300


In [306]:
#rename the injuries columns to make them easier to read
final_data = final_data.rename(columns={"inj_tot_f": "Total_Fatal_Injuries", 
                                        "inj_tot_s":"Total_Serious_Injuries",
                                        "inj_tot_m":"Total_Minor_Injuries",
                                        "inj_tot_n":'Total_Uninjured',
                                        "inj_tot_t":"Total_Injuries_Flight"})

#fill in 0s when there wasn't an injury in that category
final_data.update(final_data[['Total_Fatal_Injuries','Total_Serious_Injuries',
                              'Total_Minor_Injuries','Total_Uninjured',
                              'Total_Injuries_Flight','inj_f_grnd',
                              'inj_m_grnd','inj_s_grnd']].fillna(0))
final_data.head()

,ev_id,acft_make,acft_model,cert_max_gr_wt,acft_category,damage,far_part,afm_hrs_last_insp,type_fly,dprt_city,dprt_state,rwy_len,rwy_width,ev_type,ev_city,ev_state,ev_country,ev_highest_injury,inj_f_grnd,inj_m_grnd,inj_s_grnd,Total_Fatal_Injuries,Total_Minor_Injuries,Total_Uninjured,Total_Serious_Injuries,Total_Injuries_Flight,sky_cond_ceil,sky_cond_nonceil,wx_int_precip,phase_flt_spec
0,20001204X00000,CESSNA,207,3800.0,AIR,SUBS,135,75.0,UNK,BETHEL,AK,NaN,NaN,ACC,QUINHAGAK,AK,USA,MINR,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,BKN,UNK,UNK,APPROACH
1,20001204X00001,BOEING,747-100,750000.0,AIR,MINR,121,113.0,UNK,CHITOSE,JA,11800.0,150.0,INC,FAIRBANKS,AK,USA,NONE,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,NONE,SCAT,UNK,LANDING
2,20001204X00002,PIPER,PA-31-350,7369.0,AIR,SUBS,135,32.0,UNK,CHENEGA BAY,AK,NaN,NaN,ACC,ANCHORAGE,AK,USA,NONE,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,OVC,SCAT,UNK,UNKNOWN
3,20001204X00003,CESSNA,172,2300.0,AIR,SUBS,091,40.0,PERS,NaN,NaN,6398.0,150.0,ACC,BETHEL,AK,USA,NONE,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,BKN,UNK,LGT,UNKNOWN
4,20001204X00004,CESSNA,207,3800.0,AIR,SUBS,135,49.0,UNK,NaN,AK,2610.0,40.0,ACC,CHEVAK,AK,USA,NONE,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,BKN,UNK,UNK,DESCENT


In [307]:
final_data.dropna(subset=['cert_max_gr_wt','afm_hrs_last_insp',
                          'rwy_len','rwy_width'],inplace=True)

In [308]:
final_data = final_data.reset_index(drop=True)

In [309]:
final_data.update(final_data.fillna("UNK"))


In [310]:
phase_df = final_data.copy()

In [311]:
#we want to account for ALL injuries. This includes injuries on the ground as well as passangers
#Here we will make a new column that shows total injuries including ground ones
final_data['Total_Injuries_Ground'] = final_data['inj_f_grnd']+final_data['inj_m_grnd']+final_data['inj_s_grnd']
final_data['Total_Injuries'] = final_data['Total_Injuries_Ground']+final_data['Total_Injuries_Flight']
final_data.head()

,ev_id,acft_make,acft_model,cert_max_gr_wt,acft_category,damage,far_part,afm_hrs_last_insp,type_fly,dprt_city,dprt_state,rwy_len,rwy_width,ev_type,ev_city,ev_state,ev_country,ev_highest_injury,inj_f_grnd,inj_m_grnd,inj_s_grnd,Total_Fatal_Injuries,Total_Minor_Injuries,Total_Uninjured,Total_Serious_Injuries,Total_Injuries_Flight,sky_cond_ceil,sky_cond_nonceil,wx_int_precip,phase_flt_spec,Total_Injuries_Ground,Total_Injuries
0,20001204X00001,BOEING,747-100,750000.0,AIR,MINR,121,113.0,UNK,CHITOSE,JA,11800.0,150.0,INC,FAIRBANKS,AK,USA,NONE,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,NONE,SCAT,UNK,LANDING,0.0,0.0
1,20001204X00003,CESSNA,172,2300.0,AIR,SUBS,091,40.0,PERS,UNK,UNK,6398.0,150.0,ACC,BETHEL,AK,USA,NONE,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,BKN,UNK,LGT,UNKNOWN,0.0,0.0
2,20001204X00004,CESSNA,207,3800.0,AIR,SUBS,135,49.0,UNK,UNK,AK,2610.0,40.0,ACC,CHEVAK,AK,USA,NONE,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,BKN,UNK,UNK,DESCENT,0.0,0.0
3,20001204X00006,BEECH,300,14100.0,AIR,DEST,091,3.0,EXEC,GREENEVILLE,SC,5500.0,100.0,ACC,CULLMAN,AL,USA,FATL,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,BKN,UNK,MOD,APPROACH,0.0,2.0
4,20001204X00008,AERO COMMANDER,560A,6000.0,AIR,DEST,091,13.0,PERS,UNK,UNK,3800.0,36.0,ACC,BELLEVIEW,FL,USA,FATL,0.0,0.0,0.0,2.0,0.0,0.0,2.0,4.0,NONE,CLER,UNK,APPROACH,0.0,4.0


In [312]:
final_data['Injury'] = np.where(final_data['Total_Injuries'] >0,1,0)
injuries = final_data["Injury"].value_counts().reset_index()
injuries.head(3)

,index,Injury
0,1,18750
1,0,16677


In [313]:
final_df = final_data.copy()
#Since we added up all of our injuries we don't need the other columns that include injury count since it will be colinear to our prediction variable
final_df = final_df.drop(['Total_Fatal_Injuries','Total_Serious_Injuries','Total_Minor_Injuries',
                          'Total_Uninjured','Total_Injuries_Flight','inj_f_grnd','inj_m_grnd',
                          'inj_s_grnd','Total_Injuries_Ground',"Total_Injuries","ev_highest_injury",
                          "ev_id","dprt_city"],axis = 1)
final_df = final_df.reset_index(drop=True)

# Final Dataframe for predicting total injury

In [285]:
df = final_df.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35427 entries, 0 to 35426
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   acft_make          35427 non-null  object 
 1   acft_model         35427 non-null  object 
 2   cert_max_gr_wt     35427 non-null  float64
 3   acft_category      35427 non-null  object 
 4   damage             35427 non-null  object 
 5   far_part           35427 non-null  object 
 6   afm_hrs_last_insp  35427 non-null  float64
 7   type_fly           35427 non-null  object 
 8   dprt_state         35427 non-null  object 
 9   rwy_len            35427 non-null  float64
 10  rwy_width          35427 non-null  float64
 11  ev_type            35427 non-null  object 
 12  ev_city            35427 non-null  object 
 13  ev_state           35427 non-null  object 
 14  ev_country         35427 non-null  object 
 15  sky_cond_ceil      35427 non-null  object 
 16  sky_cond_nonceil   354

In [286]:
X = df.drop("Injury", axis = 1).copy()
y = df["Injury"].copy()

In [287]:
#One hot encode specific columns without standardizing and scaling continuous variables
from sklearn.preprocessing import OneHotEncoder
categorical_features = ['acft_make', 'acft_model', 'acft_category', 'damage','far_part', 'type_fly',
                        'dprt_state','ev_type','ev_city', 'ev_state','ev_country', 'sky_cond_ceil', 'sky_cond_nonceil',
                        'wx_int_precip', 'phase_flt_spec']

ohe = OneHotEncoder()

X_object = X.select_dtypes('object')
ohe.fit(X_object)

codes = ohe.transform(X_object).toarray()
feature_names = ohe.get_feature_names(categorical_features)

X = pd.concat([X.select_dtypes(exclude='object'), 
               pd.DataFrame(codes,columns=feature_names).astype(int)], axis=1)

In [261]:
X.reset_index(drop = True, inplace=True)

In [20]:
# from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer

# numeric_features = ['cert_max_gr_wt',"afm_hrs_last_insp","rwy_width",'rwy_len']

# numeric_transformer = Pipeline(steps=[('scaler', StandardScaler(with_mean=False))])

# categorical_features = ['acft_make', 'acft_model', 'acft_category','damage',
#                         'type_fly', 'dprt_state','ev_type','ev_state',
#                         'sky_cond_ceil', 'sky_cond_nonceil','wx_int_precip',
#                         'phase_flt_spec','ev_city','ev_country','far_part']


# categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(drop="first"))])


# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)])

# the_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

In [47]:
# X = the_pipeline.fit_transform(X)
# X

<35427x18468 sparse matrix of type '<class 'numpy.float64'>'
	with 587128 stored elements in Compressed Sparse Row format>

In [48]:
# LE = LabelEncoder()
# LE.fit(y)
# y = LE.transform(y)
# y

array([0, 0, 0, ..., 1, 1, 1])

In [239]:
#  X_train, X_test = X.iloc[train_index], X.iloc[test_index]

#  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [289]:
from sklearn.model_selection import StratifiedShuffleSplit 
cv = StratifiedShuffleSplit(n_splits=2,test_size=0.10, random_state=42)

In [290]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt
import time
lr_clf = LogisticRegression(class_weight="balanced",solver='liblinear', penalty="l2",max_iter=1000,random_state=42)
iter_num=0
for train_indices, test_indices in cv.split(X,y): 
#     start = time.time()
#     elapsed_time = (time.time() - start)
    X_train = X.iloc[train_indices]
    y_train = y.iloc[train_indices]
    
    X_test = X.iloc[test_indices]
    y_test = y.iloc[test_indices]
    lr_clf.fit(X_train,y_train)  # train object

    y_hat = lr_clf.predict(X_test) # get test set predictions
    print("====Iteration",iter_num," ====")
    acc = mt.accuracy_score(y_test,y_hat)
    conf = mt.confusion_matrix(y_test,y_hat)
    print('accuracy:', acc )
    print(conf )
#     print('CV Time: ', elapsed_time)
    iter_num+=1

/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


====Iteration 0  ====
accuracy: 0.8159751622918431
[[1584   84]
 [ 568 1307]]
====Iteration 1  ====
accuracy: 0.8173863957098504
[[1535  133]
 [ 514 1361]]


# Dataframe for predicting broad phase of flight

In [314]:
phase_df["phase_flt_spec"].value_counts()

LANDING        16743
TAKEOFF         5776
APPROACH        3300
DESCENT         2995
MANEUVERING     1569
CRUISE          1218
UNKNOWN         1179
CLIMB            923
TAXI             741
STANDING         393
GOAROUND         301
OTHER            162
HOVER             94
UNK               33
Name: phase_flt_spec, dtype: int64

In [315]:
#Removed columns that contain UNKNOWN and UNK
new_phase_df = phase_df[(phase_df['phase_flt_spec'] != "UNKNOWN") & (phase_df['phase_flt_spec'] != "UNK")]

In [316]:
new_phase_df["phase_flt_spec"].value_counts()

LANDING        16743
TAKEOFF         5776
APPROACH        3300
DESCENT         2995
MANEUVERING     1569
CRUISE          1218
CLIMB            923
TAXI             741
STANDING         393
GOAROUND         301
OTHER            162
HOVER             94
Name: phase_flt_spec, dtype: int64

In [318]:
new_phase_df.reset_index(drop=True,inplace=True)

In [320]:
del new_phase_df["ev_id"]

For classifying the broad phase of flight we want to include all the different typles of injury columns

In [322]:
new_phase_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34215 entries, 0 to 34214
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   acft_make               34215 non-null  object 
 1   acft_model              34215 non-null  object 
 2   cert_max_gr_wt          34215 non-null  float64
 3   acft_category           34215 non-null  object 
 4   damage                  34215 non-null  object 
 5   far_part                34215 non-null  object 
 6   afm_hrs_last_insp       34215 non-null  float64
 7   type_fly                34215 non-null  object 
 8   dprt_city               34215 non-null  object 
 9   dprt_state              34215 non-null  object 
 10  rwy_len                 34215 non-null  float64
 11  rwy_width               34215 non-null  float64
 12  ev_type                 34215 non-null  object 
 13  ev_city                 34215 non-null  object 
 14  ev_state                34215 non-null

In [323]:
new_phase_df

,acft_make,acft_model,cert_max_gr_wt,acft_category,damage,far_part,afm_hrs_last_insp,type_fly,dprt_city,dprt_state,rwy_len,rwy_width,ev_type,ev_city,ev_state,ev_country,ev_highest_injury,inj_f_grnd,inj_m_grnd,inj_s_grnd,Total_Fatal_Injuries,Total_Minor_Injuries,Total_Uninjured,Total_Serious_Injuries,Total_Injuries_Flight,sky_cond_ceil,sky_cond_nonceil,wx_int_precip,phase_flt_spec
0,BOEING,747-100,750000.0,AIR,MINR,121,113.0,UNK,CHITOSE,JA,11800.0,150.0,INC,FAIRBANKS,AK,USA,NONE,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,NONE,SCAT,UNK,LANDING
1,CESSNA,207,3800.0,AIR,SUBS,135,49.0,UNK,UNK,AK,2610.0,40.0,ACC,CHEVAK,AK,USA,NONE,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,BKN,UNK,UNK,DESCENT
2,BEECH,300,14100.0,AIR,DEST,091,3.0,EXEC,GREENEVILLE,SC,5500.0,100.0,ACC,CULLMAN,AL,USA,FATL,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,BKN,UNK,MOD,APPROACH
3,AERO COMMANDER,560A,6000.0,AIR,DEST,091,13.0,PERS,UNK,UNK,3800.0,36.0,ACC,BELLEVIEW,FL,USA,FATL,0.0,0.0,0.0,2.0,0.0,0.0,2.0,4.0,NONE,CLER,UNK,APPROACH
4,GETTIS H. HUDSON,CORBAN BABY ACE,900.0,AIR,SUBS,091,10.0,PERS,UNK,UNK,1900.0,75.0,ACC,PLEASANT VIEW,TN,USA,NONE,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NONE,OVCT,UNK,TAKEOFF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34210,DEHAVILLAND CANADA,DHC-1 CHIPMUNK X,2080.0,AIR,SUBS,091,1.0,PERS,UNK,UNK,2700.0,80.0,ACC,LEBANON,TN,USA,SERS,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,UNK,FEW,UNK,DESCENT
34211,BOMBARDIER INC,CL-600-2B16,48300.0,AIR,DEST,091,17.0,PERS,COEUR D'ALENE,ID,7001.0,100.0,ACC,TRUCKEE,CA,USA,FATL,0.0,0.0,0.0,6.0,0.0,0.0,0.0,6.0,BKN,UNK,UNK,APPROACH
34212,STEPHEN L KRUER,SUPERSTOL,1320.0,AIR,SUBS,091,1.0,PERS,SALEM,IN,600.0,100.0,ACC,FLOYDS KNOBS,IN,USA,SERS,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NONE,CLER,UNK,GOAROUND
34213,CESSNA,140,1450.0,AIR,SUBS,091,44.0,PERS,UNK,UNK,4000.0,75.0,ACC,GREENVILLE,IL,USA,NONE,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,NONE,CLER,UNK,LANDING


In [325]:
new_phase_df["phase_flt_spec"].value_counts()

LANDING        16743
TAKEOFF         5776
APPROACH        3300
DESCENT         2995
MANEUVERING     1569
CRUISE          1218
CLIMB            923
TAXI             741
STANDING         393
GOAROUND         301
OTHER            162
HOVER             94
Name: phase_flt_spec, dtype: int64

|Phase of Flight||Code|
|----||----|
|APPROACH||0|
|APPROACH||1|
|APPROACH||2|
|APPROACH||3|
|APPROACH||4|


- LANDING        16743
- TAKEOFF         5776
- APPROACH        0
- DESCENT         2995
- MANEUVERING     1569
- CRUISE          2
- CLIMB            1
- TAXI             741
- STANDING         393
- GOAROUND         301
- OTHER            162
- HOVER             94

In [ ]:
new_phase_df["phase_flt_spec"] = new_phase_df["phase_flt_spec"].replace{"LANDING":}